In [1]:
import jax
import jax.numpy as jnp
from jax import jit, vmap, grad
from jax import random
import optax
import numpy as np
import pickle

In [2]:
def init_network_params(sizes, key=random.PRNGKey(4)):
    def random_layer_params(m, n, key, scale=1e-2):
        w_key, b_key = random.split(key)
        return scale * random.uniform(w_key, (n, m), minval=1, maxval=3), scale * random.uniform(b_key, (n,), minval=1, maxval=3)

    keys = random.split(key, len(sizes))
    return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

def predict(NN, x):
    activations = x.reshape((-1, ))
    for w, b in NN[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = jax.nn.sigmoid(outputs)
    final_w, final_b = NN[-1]
    logit = jnp.dot(final_w, activations) + final_b
    logit = 1/(1+jnp.exp(-logit))
    return logit.squeeze()

vpredict = vmap(predict, in_axes=(None, 0))
gpredict = grad(predict, argnums=1)
vgpredict = vmap(gpredict, in_axes=(None, 0))

In [3]:
def pi(x):
    return (1 / jnp.sqrt(2 * jnp.pi)) * jnp.exp(-0.5 * x**2)
vpi = vmap(pi, in_axes=(0,))

def pi_pre(NN, x):
    result = gpredict(NN, x)
    return result

@jit
def obj(NN, x_batch):
    def obj1(NN, x):
        return pi(x) - pi_pre(NN, x)
    vobj1 = vmap(obj1, in_axes=(None, 0))

    result = vobj1(NN, x_batch)
    result = jnp.square(result)
    result = jnp.mean(result)
    return result
jgo = jit(grad(obj))

@jit
def evaluation(NN, test_zu):
    z_batch, u_batch = test_zu[:, 0], test_zu[:, 1]
    result = jnp.square(vpredict(NN, z_batch) - u_batch)
    result = jnp.mean(result)
    return result

In [ ]:
LAYER_SIZES = [1, 10, 10, 10, 1]
NN = init_network_params(LAYER_SIZES, random.PRNGKey(15))

optimizer= optax.adam(0.001)
opt_state = optimizer.init(NN)
test_zu = np.load('/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/test_zu.npy')
lowest = evaluation(NN, test_zu)
Lloss = []
Llowest = []
for epoch in range(2000000):
    x_batch = np.random.uniform(-10, 10, (10000, 1)) #1000 refers to batch size

    grads = grad(obj)(NN, x_batch)
    updates, opt_state = optimizer.update(grads, opt_state)
    NN = optax.apply_updates(NN, updates)

    Lloss.append(obj(NN, x_batch).item())
    Llowest.append(lowest.item())
    print(f"Epoch: {epoch}, loss:{obj(NN, x_batch)}, {lowest}")

    if  evaluation(NN, test_zu)<lowest:
        lowest = evaluation(NN, test_zu)
        NN_best = NN

    if epoch % 10000 == 0:
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/Interval 10/Lloss.npy', Lloss)
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/Interval 10/Llowest.npy', Llowest)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/Interval 10/NN (best).pkl', 'wb') as f:
            pickle.dump(NN_best, f)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/Interval 10/NN_{epoch}.pkl', 'wb') as f:
            pickle.dump(NN, f)

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch: 1768277, loss:2.861733072734296e-09, 1.9556147812238578e-09
Epoch: 1768278, loss:2.7096749288801902e-09, 1.9556147812238578e-09
Epoch: 1768279, loss:2.495203599295337e-09, 1.9556147812238578e-09
Epoch: 1768280, loss:2.5105950651749254e-09, 1.9556147812238578e-09
Epoch: 1768281, loss:2.302690926825335e-09, 1.9556147812238578e-09
Epoch: 1768282, loss:1.7583129396925301e-09, 1.9556147812238578e-09
Epoch: 1768283, loss:1.651973002836371e-09, 1.9556147812238578e-09
Epoch: 1768284, loss:2.011791400136076e-09, 1.9556147812238578e-09
Epoch: 1768285, loss:2.7099966715127266e-09, 1.9556147812238578e-09
Epoch: 1768286, loss:3.51074769255888e-09, 1.9556147812238578e-09
Epoch: 1768287, loss:5.0443493826435315e-09, 1.9556147812238578e-09
Epoch: 1768288, loss:7.5634494223209e-09, 1.9556147812238578e-09
Epoch: 1768289, loss:1.049676079389883e-08, 1.9556147812238578e-09
Epoch: 1768290, loss:2.007375421442248e-08, 1.9556147812238578e-09
Epoch: 1768291, loss:2.9903429066